# 규칙 파싱 테스트 노트북
## 주택임대차 RAG - 시행령/시행규칙/대법원규칙 파싱

### 1. 필요 패키지 임포트

In [1]:
from pathlib import Path
from docx import Document
import pandas as pd
import re
import uuid
from datetime import datetime
from collections import Counter

print("✅ 패키지 임포트 완료")

✅ 패키지 임포트 완료


### 2. 카테고리 정의

In [2]:
def get_law_category():
    """주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전"""
    return {
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2,
            '반환보증': 2, 'HUG': 1, '보증보험': 2
        },
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1
        },
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },
        '임대료_증감': {
            '차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2,
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1
        },
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3,
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2
        },
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1
        },
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2
        },
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2,
            '공동상속': 2, '반환청구권': 2
        }
    }

print("✅ 카테고리 정의 완료")
print(f"총 {len(get_law_category())}개 카테고리")

✅ 카테고리 정의 완료
총 9개 카테고리


### 3. 카테고리 분류 함수

In [3]:
def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환"""
    category_keywords = get_law_category()
    category_scores = {}
    
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword)
            score += count * weight
        if score > 0:
            category_scores[category] = score
    
    sorted_categories = sorted(category_scores.items(), key=lambda x: x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    
    if not all_categories:
        all_categories = ["기타"]
    
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories

# 테스트
test_text = "보증금 반환과 우선변제권에 대한 내용입니다. 확정일자를 받아야 합니다."
print("테스트 결과:", categorize_content(test_text, top_k=3))

테스트 결과: ['보증금_대항력']


### 4. 경로 설정

In [4]:
# 현재 노트북 기준 경로 설정
BASE_DIR = Path.cwd().parents[1]  # 노트북이 src/ 하위에 있다고 가정

DATA_DIR = BASE_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
CSV_DIR = PROCESSED_DIR / "csv"
LOG_DIR = PROCESSED_DIR / "log"

RULE_RAW_DIR = RAW_DIR / "rule"
RULE_CSV_PATH = CSV_DIR / "rule.csv"

# 디렉토리 생성
CSV_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)
RULE_RAW_DIR.mkdir(parents=True, exist_ok=True)

print("📁 경로 설정 완료")
print(f"Raw 디렉토리: {RULE_RAW_DIR}")
print(f"CSV 저장 경로: {RULE_CSV_PATH}")

📁 경로 설정 완료
Raw 디렉토리: c:\project-AI2\실프로젝트\data\raw\rule
CSV 저장 경로: c:\project-AI2\실프로젝트\data\processed\csv\rule.csv


### 5. 파싱 함수 정의

In [5]:
ARTICLE_PATTERN = re.compile(r"(제\s*\d+조(?:의\d+)?)")

def extract_article_title(text):
    """조문에서 제목 추출 (괄호 안의 내용)"""
    match = re.search(r'\(([^)]+)\)', text)
    if match:
        return match.group(1)
    return ""

from langchain_text_splitters import RecursiveCharacterTextSplitter

def parse_rule_docx(file_path, src_title, priority, effective_date):
    """규칙 DOCX 파일을 파싱하여 적절한 크기로 청킹"""
    try:
        doc = Document(file_path)
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return pd.DataFrame()
    
    # 전체 텍스트 수집
    full_text = "\n".join([para.text.strip() for para in doc.paragraphs if para.text.strip()])
    
    # 빈 텍스트 체크
    if not full_text or len(full_text) < 10:
        print(f"⚠️ 텍스트가 너무 짧습니다: {file_path}")
        return pd.DataFrame()
    
    # RecursiveCharacterTextSplitter로 청킹
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    chunks = text_splitter.split_text(full_text)
    
    rows = []
    for idx, chunk in enumerate(chunks):
        if not chunk.strip() or len(chunk.strip()) < 10:  # 빈 청크 제외
            continue
            
        # 조문 번호 추출 (첫 번째 발견된 것)
        article_match = ARTICLE_PATTERN.search(chunk)
        article = article_match.group(1).replace(" ", "") if article_match else ""
        
        categories = categorize_content(chunk, top_k=3)
        
        rows.append({
            "id": str(uuid.uuid4()),
            "chunk_id": f"chunk_{idx:03d}",
            "text": chunk.strip(),
            "src_title": src_title,
            "article": article,
            "title": extract_article_title(chunk) if article else "",
            "category": ", ".join(categories),
            "priority": priority,
            "effective_date": effective_date,
            "source": str(file_path),
            "parsed_at": datetime.now().isoformat()
        })
    
    return pd.DataFrame(rows)

print("✅ 파싱 함수 정의 완료")

✅ 파싱 함수 정의 완료


### 6. 파일 검색 및 파싱 실행

In [6]:
# DOCX 파일 찾기
docx_files = list(RULE_RAW_DIR.glob("*.docx"))
docx_files = [f for f in docx_files if not f.name.startswith('~$')]  # 임시 파일 제외

print("=" * 60)
print("규칙 DOCX 파일 파싱 시작")
print("=" * 60)

if not docx_files:
    print(f"\n⚠️  규칙 파일이 없습니다.")
    print(f"📁 다음 경로에 DOCX 파일을 추가해주세요:")
    print(f"   {RULE_RAW_DIR}")
else:
    print(f"\n📂 발견된 DOCX 파일: {len(docx_files)}개")
    for f in docx_files:
        print(f"   - {f.name}")

규칙 DOCX 파일 파싱 시작

📂 발견된 DOCX 파일: 1개
   - 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx


In [7]:
# 파싱 실행
all_dfs = []

for docx_file in docx_files:
    print(f"\n🔄 파싱 중: {docx_file.name}")
    
    filename = docx_file.stem
    
    # 파일명 패턴 분석
    if "시행령" in filename:
        match = re.search(r'(\d+)_(\d{8})', filename)
        if match:
            law_no = match.group(1)
            date = match.group(2)
            src_title = f"주택임대차보호법 시행령(대통령령)(제{law_no}호)({date})"
            effective_date = date
        else:
            src_title = "주택임대차보호법 시행령(대통령령)"
            effective_date = "2026-01-02"
        priority = 2
        
    elif "시행규칙" in filename:
        src_title = "주택임대차보호법 시행규칙"
        priority = 3
        effective_date = "2026-01-02"

    elif "법무부령" in filename:
        match = re.search(r'(\d+)_(\d{8})', filename)
        if match:
            law_no = match.group(1)
            date = match.group(2)
            src_title = f"주택임대차계약증서상의 확정일자 부여 및 임대차 정보제공에 관한 규칙(법무부령)(제{law_no}호)({date})"
            effective_date = date
        else:
            src_title = "확정일자 관련 규칙(법무부령)"
            effective_date = "2022-02-07"
        priority = 3
        
    elif "확정일자" in filename or "대법원규칙" in filename:
        match = re.search(r'(\d+)_(\d{8})', filename)
        if match:
            law_no = match.group(1)
            date = match.group(2)
            src_title = f"확정일자 대법원규칙(제{law_no}호)({date})"
            effective_date = date
        else:
            src_title = "확정일자 대법원규칙"
            effective_date = "2021-06-10"
        priority = 3
        
    else:
        src_title = filename
        priority = 3
        effective_date = "2026-01-02"
    
    df = parse_rule_docx(
        file_path=docx_file,
        src_title=src_title,
        priority=priority,
        effective_date=effective_date
    )
    
    if not df.empty:
        all_dfs.append(df)
        print(f"   ✅ {len(df)} 조문 파싱 완료")
    else:
        print(f"   ⚠️ 파싱된 조문이 없습니다.")

print("\n" + "=" * 60)
print("파싱 완료")
print("=" * 60)


🔄 파싱 중: 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙(대법원규칙)(제02986호)(20210610).docx
   ✅ 4 조문 파싱 완료

파싱 완료


### 7. 결과 저장 및 통계

In [8]:
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(RULE_CSV_PATH, index=False, encoding="utf-8-sig")
    
    print(f"✅ rule.csv 생성 완료!")
    print(f"📊 총 {len(final_df)} 조문 저장")
    print(f"📁 저장 경로: {RULE_CSV_PATH}")
else:
    print("❌ 파싱된 데이터가 없습니다.")

✅ rule.csv 생성 완료!
📊 총 4 조문 저장
📁 저장 경로: c:\project-AI2\실프로젝트\data\processed\csv\rule.csv


In [9]:
# 규칙별 통계
if all_dfs:
    print("\n📈 규칙별 통계:")
    stats = final_df.groupby('src_title').size()
    for law, count in stats.items():
        print(f"   - {law}: {count}개")


📈 규칙별 통계:
   - 확정일자 대법원규칙: 4개


In [10]:
# 카테고리별 통계
if all_dfs:
    print("\n📊 카테고리별 통계:")
    all_categories = []
    for cats in final_df['category']:
        all_categories.extend([c.strip() for c in cats.split(',')])
    
    category_counts = Counter(all_categories)
    for cat, count in category_counts.most_common():
        print(f"   - {cat}: {count}개")


📊 카테고리별 통계:
   - 행정절차: 4개
   - 보증금_대항력: 4개
   - 임대료_증감: 2개
   - 권리_리스크: 1개


### 8. 결과 미리보기

In [11]:
# 첫 5개 행 확인
if all_dfs:
    print("\n📋 CSV 미리보기 (첫 5개):")
    display(final_df.head())


📋 CSV 미리보기 (첫 5개):


,id,chunk_id,text,src_title,article,title,category,priority,effective_date,source,parsed_at
0,59c1c89f-3181-4871-88c2-dfd2bd496ab5,chunk_000,주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙\n[시행 2021. 6....,확정일자 대법원규칙,제1조,부동산등기과,"행정절차, 보증금_대항력",3,2021-06-10,c:\project-AI2\실프로젝트\data\raw\rule\주택임대차계약증서의 ...,2026-01-21T12:40:52.830837
1,e4656595-1684-4caf-b2a4-836704a631dd,chunk_001,③ 제2항제2호의 자격자대리인은「부동산등기규칙」제68조에 따른 사용자(이용)등록정보...,확정일자 대법원규칙,제68조,이용,"보증금_대항력, 행정절차, 임대료_증감",3,2021-06-10,c:\project-AI2\실프로젝트\data\raw\rule\주택임대차계약증서의 ...,2026-01-21T12:40:52.830837
2,63ebe5c1-1ed3-4764-a875-4ab83028edd3,chunk_002,1. 해당 주택의 임대인ㆍ임차인\n2. 해당 주택의 소유자\n3. 해당 주택 또는 ...,확정일자 대법원규칙,제3조의2,확정일자 정보제공의 요청방법,"행정절차, 보증금_대항력, 임대료_증감",3,2021-06-10,c:\project-AI2\실프로젝트\data\raw\rule\주택임대차계약증서의 ...,2026-01-21T12:40:52.830837
3,dc124770-b8a2-4991-9485-552df4e7a228,chunk_003,1. 「국민기초생활 보장법」제2조제2호에 따른 수급자\n2. 「독립유공자예우에 관한...,확정일자 대법원규칙,제2조,선순위자만 해당된다,"권리_리스크, 행정절차, 보증금_대항력",3,2021-06-10,c:\project-AI2\실프로젝트\data\raw\rule\주택임대차계약증서의 ...,2026-01-21T12:40:52.830837


In [12]:
# 특정 조문 확인
if all_dfs:
    print("\n🔍 특정 조문 샘플:")
    sample = final_df[final_df['article'].str.contains('제1조', na=False)].iloc[0]
    print(f"조문: {sample['article']}")
    print(f"제목: {sample['title']}")
    print(f"카테고리: {sample['category']}")
    print(f"우선순위: {sample['priority']}")
    print(f"본문(앞 200자): {sample['text'][:200]}...")


🔍 특정 조문 샘플:
조문: 제1조
제목: 부동산등기과
카테고리: 행정절차, 보증금_대항력
우선순위: 3
본문(앞 200자): 주택임대차계약증서의 확정일자 부여 및 정보제공에 관한 규칙
[시행 2021. 6. 10.] [대법원규칙 제2986호, 2021. 5. 27., 타법개정]
법원행정처(부동산등기과) 02-3480-1394
제1조(목적) 이 규칙은 지방법원 및 그 지원과 등기소가「주택임대차 보호법」(이하 “법”이라 한다)이 정하고 있는 확정일자 부여 등의 업무를 처리함에 있어 ...


### 9. 컬럼 정보 확인

In [13]:
if all_dfs:
    print("\n📋 데이터프레임 정보:")
    print(final_df.info())
    print("\n컬럼 목록:")
    print(final_df.columns.tolist())


📋 데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              4 non-null      object
 1   chunk_id        4 non-null      object
 2   text            4 non-null      object
 3   src_title       4 non-null      object
 4   article         4 non-null      object
 5   title           4 non-null      object
 6   category        4 non-null      object
 7   priority        4 non-null      int64 
 8   effective_date  4 non-null      object
 9   source          4 non-null      object
 10  parsed_at       4 non-null      object
dtypes: int64(1), object(10)
memory usage: 480.0+ bytes
None

컬럼 목록:
['id', 'chunk_id', 'text', 'src_title', 'article', 'title', 'category', 'priority', 'effective_date', 'source', 'parsed_at']
